<a href="https://colab.research.google.com/github/Rushikesh419/Machine-learning/blob/main/recommedation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19.
! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  1530k      0 --:--:-- --:--:-- --:--:-- 1530k


In [2]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [4]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)


The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [5]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")


Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [10]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)

In [12]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()

        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()

        #--- Producing new continuous IDs for users and movies ---

        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])


        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)


In [13]:

num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0403, 0.0105, 0.0188,  ..., 0.0153, 0.0399, 0.0487],
        [0.0235, 0.0466, 0.0234,  ..., 0.0253, 0.0494, 0.0216],
        [0.0092, 0.0460, 0.0339,  ..., 0.0458, 0.0350, 0.0166],
        ...,
        [0.0216, 0.0156, 0.0242,  ..., 0.0456, 0.0392, 0.0163],
        [0.0039, 0.0031, 0.0431,  ..., 0.0203, 0.0267, 0.0407],
        [0.0173, 0.0141, 0.0207,  ..., 0.0480, 0.0447, 0.0169]])
item_factors.weight tensor([[0.0324, 0.0154, 0.0122,  ..., 0.0387, 0.0360, 0.0340],
        [0.0256, 0.0101, 0.0339,  ..., 0.0355, 0.0416, 0.0074],
        [0.0360, 0.0036, 0.0188,  ..., 0.0094, 0.0285, 0.0170],
        ...,
        [0.0448, 0.0021, 0.0401,  ..., 0.0033, 0.0245, 0.0222],
        [0.0484, 0.0433, 0.0304,  ..., 0.0474, 0.0037, 0.0083],
        [0.0476, 0.0346, 0.0203,  ..., 0.0193, 0.0144, 0.0008]])


In [15]:


for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
        # Removed the cuda check as it is False in the user's environment
        #if cuda:
        # The following lines were indented incorrectly
        x, y = x.cuda() if cuda else x, y.cuda() if cuda else y # move data to GPU if available
        optimizer.zero_grad()
        outputs = model(x)
        loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-15-cb13e4fa8b97>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.065080388548411
iter #1 Loss: 4.7455992184314635
iter #2 Loss: 2.475532400275245
iter #3 Loss: 1.7218897309400103
iter #4 Loss: 1.3462880938337538
iter #5 Loss: 1.1291174230539254
iter #6 Loss: 0.9920042400735284
iter #7 Loss: 0.9006651128609169
iter #8 Loss: 0.8375712981457033
iter #9 Loss: 0.792587477846194
iter #10 Loss: 0.759675044963505
iter #11 Loss: 0.7349073519637137
iter #12 Loss: 0.7163442147867328
iter #13 Loss: 0.7015028681776245
iter #14 Loss: 0.6907203250049334
iter #15 Loss: 0.681976584651446
iter #16 Loss: 0.675091048409491
iter #17 Loss: 0.6697254832928556
iter #18 Loss: 0.6658929109497724
iter #19 Loss: 0.6630588244590058
iter #20 Loss: 0.6603477779908229
iter #21 Loss: 0.6588931863153646
iter #22 Loss: 0.657807159960875
iter #23 Loss: 0.6566987984298449
iter #24 Loss: 0.6557741224917059
iter #25 Loss: 0.6548723629510342
iter #26 Loss: 0.6543344887077506
iter #27 Loss: 0.6534975932409921
iter #28 Loss: 0.6524294373498956
iter #29 Loss: 0.6510172935987

In [16]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)


user_factors.weight tensor([[ 1.4255,  1.3223,  1.6096,  ...,  0.6854,  1.0390,  1.2214],
        [ 1.2135,  2.1529,  0.6592,  ...,  1.3154,  0.8281,  0.5681],
        [-0.9988,  1.3604, -1.8855,  ...,  0.6481,  2.5881,  0.6947],
        ...,
        [ 0.6405,  0.7532, -0.5791,  ...,  2.1697,  0.1780,  2.2141],
        [ 1.7486,  0.8629,  1.0245,  ...,  0.8573,  0.2337,  0.6440],
        [ 1.2867,  0.1874,  0.5512,  ...,  0.9927,  1.2720,  1.2099]])
item_factors.weight tensor([[ 0.3525,  0.4137,  0.8259,  ...,  0.4059,  0.5460,  0.4612],
        [ 0.5498,  0.4223,  0.6299,  ...,  0.4648,  0.1308, -0.1203],
        [ 0.5987,  0.1724,  0.2652,  ...,  0.4460,  0.4004,  0.4355],
        ...,
        [ 0.3595,  0.3051,  0.3536,  ...,  0.3169,  0.3374,  0.3367],
        [ 0.4439,  0.3818,  0.4272,  ...,  0.4437,  0.3988,  0.4041],
        [ 0.4458,  0.3758,  0.4167,  ...,  0.4168,  0.4121,  0.3984]])


In [17]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()


In [18]:
len(trained_movie_embeddings) # unique movie factor weights


9724

In [19]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)


In [20]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
    print("Cluster #{}".format(cluster))
    movs = []
    # Find movie indices belonging to the current cluster
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        movid = train_set.idx2movieid[movidx]
        # Check how many ratings this movie has
        rat_count = len(ratings_df.loc[ratings_df['movieId'] == movid])
        movs.append((movie_names[movid], rat_count))
    # Sort movies by rating count in descending order and print top 10
    for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
        print("\t", mov[0])

Cluster #0
	 Die Hard: With a Vengeance (1995)
	 Rock, The (1996)
	 Home Alone (1990)
	 Cast Away (2000)
	 Mummy, The (1999)
	 Face/Off (1997)
	 Liar Liar (1997)
	 Charlie's Angels (2000)
	 Bruce Almighty (2003)
	 Coneheads (1993)
Cluster #1
	 Ace Ventura: Pet Detective (1994)
	 Honey, I Shrunk the Kids (1989)
	 Lost World: Jurassic Park, The (1997)
	 Hot Shots! Part Deux (1993)
	 Arachnophobia (1990)
	 Batman & Robin (1997)
	 Indiana Jones and the Kingdom of the Crystal Skull (2008)
	 Multiplicity (1996)
	 Swordfish (2001)
	 Pretty in Pink (1986)
Cluster #2
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Matrix, The (1999)
	 Braveheart (1995)
	 Terminator 2: Judgment Day (1991)
	 Schindler's List (1993)
	 Toy Story (1995)
	 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
	 Lord of the Rings: The Fellowship of the Ring, The (2001)
	 Star Wars: Episode VI - Return of the Jedi (1983)
Cluster #3
	 GoldenEye (1995)
	 Austin Powers: The